# Script for writing frame header from Digital Journal of Astroplates 
(Digital Journal of Astroplates - https://docs.google.com/spreadsheets/d/1kTneTpxyJbzmQ_b8l1NdOa5YqJBe4Iod/edit#gid=1243813143) 

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import astropy
from astropy.io import fits
import os
from PIL import Image
from astropy.time import Time

# Functions

In [ ]:
def date_obs(sid_h,S1,grin_mid):
    """
    Calculate and write in fits formate and in JD the date-time of observation.
    """
    date_obs_one=""
    date_obs_jd=""
    
    if sid_h.value>=0 and sid_h.value<=12:

        if S1>=12 and S1<24: 
            delta = sid_h.value - S1  
            if abs(delta) > 8: #before mid
                delta = delta%24
                date = grin_mid - delta*u.hour
                date_obs_one = date.fits
                date_obs_jd = date.jd
            else: #after mid
                date = grin_mid - delta*u.hour
                date_obs_one = date.fits
                date_obs_jd = date.jd

        elif S1>=0 and S1<=12:
            delta = sid_h.value - S1
            date = grin_mid - delta*u.hour
            date_obs_one = date.fits
            date_obs_jd = date.jd

    elif sid_h.value>=12 and sid_h.value<24:

        if S1>=12 and S1<24:
            delta = sid_h.value - S1  
            date = grin_mid - delta*u.hour
            date_obs_one = date.fits
            date_obs_jd = date.jd

        elif S1>=0 and S1<=12:
            delta = S1 - sid_h.value
            if abs(delta) > 8: #after mid
                delta = delta%24
                date = grin_mid + delta*u.hour
                date_obs_one = date.fits
                date_obs_jd = date.jd
            else:  #before mid
                date = grin_mid + delta*u.hour
                date_obs_one = date.fits
                date_obs_jd = date.jd
            
    return date_obs_one, date_obs_jd

def two_month(D):
    """
    If we have observation on the night from one month to another.
    """
    D=D.split("-")
    D[0] = D[0].split(".")
    D[1] = D[1].split(".")
    if len(D[1][2]) < 4:#if we have '96' instead of '1996'
        D[1][2] = f"19{D[1][2]}"
    D1 = f"{D[1][2]}-{D[0][1]}-{D[0][0]}"
    D2 = f"{D[1][2]}-{D[1][1]}-{D[1][0]}"
    D2_Time_func = Time(f"{D2} 00:00:00") #date2 in Time format
    sid_h = ((D2_Time_func.sidereal_time('mean',longitude=longitude).value - 6)%24)*u.hour
    grin_mid = Time(f"{D1} 18:00:00")
    return sid_h, grin_mid

def one_month(D):
    """
    If we have observation on the night from one date to another in he same month.
    """
    D=D.split("-")
    D[1] = D[1].split(".")  
    if len(D[1][2]) < 4:#if we have '96' instead of '1996'
        D[1][2] = f"19{D[1][2]}"
    D1 = f"{D[1][2]}-{D[1][1]}-{D[0]}"
    D2 = f"{D[1][2]}-{D[1][1]}-{D[1][0]}"
    D2_Time_func = Time(f"{D2} 00:00:00") #date2 in Time format
    sid_h = ((D2_Time_func.sidereal_time('mean',longitude=longitude).value - 6)%24)*u.hour
    grin_mid = Time(f"{D1} 18:00:00")
    return sid_h, grin_mid

# Table edit

In [ ]:
obsrv = pd.read_excel('Цифровой_журнал_Астропластин.xlsx',sheet_name='Кометы')

a = obsrv['Идентификационный номер (номер каробки-номер пластины)'].duplicated()
#here "Идентификационный номер (номер каробки-номер пластины)" means "Identification number (box number - plate number)" 
for i,n in enumerate(a):
    if n  == True:
        print(i)
        print(obsrv['Идентификационный номер (номер каробки-номер пластины)'].iloc[i])
        print('~~~')

for i,ra in enumerate(obsrv['RA']):
    """
    In the RA column there is 'm' in some rows, but there isn't 's'. 
    Therefore let's remove 'm' from the data, cause it will prevent calculations. 
    """
    if 'm' in str(ra):
        obsrv['RA'][i] = obsrv['RA'][i].replace('m','')

    """
    Now we should convert coords from '\~h\~.\~~' to '~~:~~:\~~.\~~' format. 
    We should write ':' because iraf doesn't read coordinates with space.
    """
for i,ra in enumerate(obsrv['RA']):
    if 'h' in str(ra):
        # there may be some excess space or ',' instead of '.'
        #also we should remove 'h' by splitting and only then convert values
        
        ra = ra.replace(' ','').replace(',','.').split('h')
    
        h = ra[0]
        m = int(float(ra[1]))
        s = round((float(ra[1]) - m) * 60,3) 
        if len(h)<2:
            if m<10:
                if s<10:
                    obsrv['RA'][i] = f'0{h}:0{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'0{h}:0{m}:{s}'
                    
            else:
                if s<10:
                    obsrv['RA'][i] = f'0{h}:{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'0{h}:{m}:{s}'
                
        else:
            if m<10:
                if s<10:
                    obsrv['RA'][i] = f'{h}:0{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'{h} 0{m} {s}'
                    
            else:
                if s<10:
                    obsrv['RA'][i] = f'{h}:{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'{h}:{m}:{s}'
                            
"""
Now let's create new table of objects with only known RA
As we can see, number of unique objects with known RA is 
less than number of all unique objects. 
So we should use list of unique objs with know RA.
Now we will create table with unique objects with known RA 
for repeated objects with unsigned RA.
"""


ra_notnull_table = obsrv[obsrv['RA'].notnull()].reset_index()
#there may be coords without object names, so let's delete such rows
ra_notnull_table = ra_notnull_table[ra_notnull_table['OBJECT'].notnull()].reset_index()

for id_unk,obj_unk in enumerate(obsrv['OBJECT']):
    for id_kn,obj_kn in enumerate(ra_notnull_table['OBJECT']):
        if obj_unk == obj_unk: #if name not Nan
            if obj_unk == obj_kn:
                if obsrv['RA'][id_unk] != ra_notnull_table['RA'][id_kn]: 
                    if obsrv['RA'][id_unk] != obsrv['RA'][id_unk]: #if Nan
                        obsrv['RA'][id_unk] = ra_notnull_table['RA'][id_kn]
                    else: # if not Nan and has own value
                        obsrv['RA'][id_unk] = obsrv['RA'][id_unk]
                else: # if RA are the same
                    continue
        elif obj_unk != obj_unk:#if Nan
            continue
            
for i,dec in enumerate(obsrv['DEC   (градусы)']):
    if ',' in str(dec):
        obsrv['DEC   (градусы)'][i] = dec.replace(',','.').replace("'","")
        
for i,dec in enumerate(obsrv['DEC   (градусы)']):
    
    if dec == dec:
    
        d = int(float(dec))
        m = int((abs(float(dec)) - abs(d)) * 60)
        s = round(((abs(float(dec)) - abs(d)) * 60 - m)*60,3) 


        if '-' in str(dec):
            if abs(d) < 10:
                if m<10:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:0{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:0{m}:{s}'
                else:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:{m}:{s}'      
            else:
                if m<10:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'-{abs(d)}:0{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'-{abs(d)}:0{m}:{s}'
                else:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'-{abs(d)}:{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'-{abs(d)}:{m}:{s}'    
        else:
            if abs(d) < 10:
                if m<10:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'0{abs(d)}:0{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'0{abs(d)}:0{m}:{s}'
                else:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'0{abs(d)}:{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'0{abs(d)}:{m}:{s}'      
            else:
                if m<10:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'{abs(d)}:0{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'{abs(d)}:0{m}:{s}'
                else:
                    if s<10:
                        obsrv['DEC   (градусы)'][i] = f'{abs(d)}:{m}:0{s}'
                    else:
                        obsrv['DEC   (градусы)'][i] = f'{abs(d)}:{m}:{s}'
        
            
        
dec_notnull_table = obsrv[obsrv['DEC   (градусы)'].notnull()].reset_index()
#there may be coords without object names, so let's delete such rows
dec_notnull_table = dec_notnull_table[dec_notnull_table['OBJECT'].notnull()].reset_index()

for id_unk,obj_unk in enumerate(obsrv['OBJECT']):
    for id_kn,obj_kn in enumerate(dec_notnull_table['OBJECT']):
        if obj_unk == obj_unk: #if not Nan
            if obj_unk == obj_kn:
                if obsrv['DEC   (градусы)'][id_unk] != dec_notnull_table['DEC   (градусы)'][id_kn]: #if DEC values not identic
                    if obsrv['DEC   (градусы)'][id_unk] != obsrv['DEC   (градусы)'][id_unk]: #if Nan
                        obsrv['DEC   (градусы)'][id_unk] = dec_notnull_table['DEC   (градусы)'][id_kn]
                    else: #if not identic and not Nan
                        obsrv['DEC   (градусы)'][id_unk] = obsrv['DEC   (градусы)'][id_unk]
                else:#if DEC values identic
                    continue
        elif obj_unk != obj_unk:
            continue
            

"""
EXPOSITION

We should write expositions in seconds only! So let's convert it
if it is necessary.
"""
obsrv['exp_name'] = obsrv['exposition ']

for e,exp in enumerate(obsrv['exposition ']): 
            
    try:
        
        if obsrv['exposition '][e] == obsrv['exposition '][e]:
            obsrv['exposition '][e] = obsrv['exposition '][e].replace(",",";")
            if '*' not in str(exp):
                if ';' in str(exp):
                    obsrv['exposition '][e] = str(exp).split(';')
                    for g in range(0,len(obsrv['exposition '][e])):
                        if 's' in str(obsrv['exposition '][e][g]) and 'm' not in str(obsrv['exposition '][e][g]) and 'h' not in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('s','')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g]) * 1
                        elif 's' not in str(obsrv['exposition '][e][g]) and 'm' in str(obsrv['exposition '][e][g]) and 'h' not in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('m','')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g])*60
                        elif 's' not in str(obsrv['exposition '][e][g]) and 'm' not in str(obsrv['exposition '][e][g]) and 'h' in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('h','')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g])*3600
                        elif 's' in str(obsrv['exposition '][e][g]) and 'm' in str(obsrv['exposition '][e][g]) and 'h' not in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('s','').split('m')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g][0])*60 +float(obsrv['exposition '][e][g][1])
                        elif 's' not in str(obsrv['exposition '][e][g]) and 'm' in str(obsrv['exposition '][e][g]) and 'h' in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('m','').split('h')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g][0])*3600 +float(obsrv['exposition '][e][g][1])*60
                else:
                    if 's' in str(exp) and 'm' not in str(exp) and 'h' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('s','')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e]) * 1
                    elif 'm' in str(exp) and 's' not in str(exp) and 'h' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('m','')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e])*60
                    elif 's' in str(exp) and 'm' in str(exp) and 'h' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('s','').split('m')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e][0])*60 + float(obsrv['exposition '][e][1])
                    elif 'h' in str(exp) and 'm' not in str(exp) and 's' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('h','')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e])*3600
                    elif 'h' in str(exp) and 'm' in str(exp) and 's' not in str(exp):
                        exp = exp.replace('m','').split('h')
                        obsrv['exposition '][e] = float(exp[0])*3600 + float(exp[1])*60
    except ValueError:
        print(e)

"""
DATE-OBS
"""
lambd = (76+57/60+58/3600)*24/360
longitude = (76+57/60+57/3600)*u.degree
obsrv['DATE-OBS-ONE'] = ['']*len(obsrv) #create new column in the table. Here will be writen all dates and time of observation in fits format 
obsrv['DATE-OBS-ONE_2'] = ['']*len(obsrv) #create new column in the table. this one is necessary for cases when we have 2 nights (4 dates)
obsrv['DATE-OBS-JD'] = ['']*len(obsrv) #create new column in the table. in the header we should write date of observations in JD
obsrv['DATE-OBS-JD_2'] = ['']*len(obsrv) #create new column in the table. in the header we should write date of observations in JD/ this one is necessary for cases when we have 2 nights (4 dates)
# try:
    
for i in range(0,len(obsrv)):
    if '*' not in str(obsrv['exposition '][i]) and '*' not in str(obsrv['TIME-OBS (tзв-s)'][i]):#we do not work with polarization, so we exclude observations with * in time-obs
        if obsrv['TIME-OBS (tзв-s)'][i] == obsrv['TIME-OBS (tзв-s)'][i]: #check if value exist
            obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].replace('/','.')
            obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].replace('?','')
            obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].replace(" ","")

            """
            if we have 2 nights (4 dates)
            """

            if ';' in obsrv['DATE-OBS'][i]: 

                obsrv['TIME-OBS (tзв-s)'][i] = str(obsrv['TIME-OBS (tзв-s)'][i]).replace(',',';') #if we have comma we should replace it with ;, cause we will split by this sybol our 2 nights
                obsrv['TIME-OBS (tзв-s)'][i] = obsrv['TIME-OBS (tзв-s)'][i].split(';')
                time_1 = obsrv['TIME-OBS (tзв-s)'][i][0] #the first night (the first 2 dates)
                time_2 = obsrv['TIME-OBS (tзв-s)'][i][1] # the second night (the last 2 dates)

                #Then we have some comditions for time-obs which includ s or m ot both or any of them. 
                #We should write time in hours without m and s
                if 's' in time_1:
                    time_1 = time_1.replace('s','')
                    time_1 = time_1.split('h')
                    time_1_m = time_1[1].split('m')
                    S1_1 = float(time_1[0]) + float(time_1_m[0])/60 + float(time_1_m[1])/3600 #local sidereal time (from table)

                elif 's' not in time_1:

                    if 'm' in time_1:
                        time_1 = time_1.replace('m','').split('h')
                        S1_1 = float(time_1[0]) + float(time_1[1].split('m')[0])/60 #local sidereal time (from table)


                    elif 'm' not in str(time_1):
                        time_1 = time_1.split('h')
                        S1_1 = float(time_1[0]) + float(time_1[1])/60 #local sidereal time (from table)

                if 's' in time_2:
                    time_2 = time_2.replace('s','')
                    time_2 = time_2.split('h')
                    time_2_m = time_2[1].split('m')
                    S1_2 = float(time_2[0]) + float(time_2_m[0])/60 + float(time_2_m[1])/3600 #local sidereal time (from table)

                else:

                    if 'm' in time_2:
                        time_2 = time_2.replace('m','').split('h')
                        S1_2 = float(time_2[0]) + float(time_2[1].split('m')[0])/60 #local sidereal time (from table)
                    elif 'm' not in str(time_2):
                        time_2 = time_2.split('h')
                        S1_2 = float(time_2[0]) + float(time_2[1])/60 #local sidereal time (from table)

                obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].split(';')
                
                D_1 = obsrv['DATE-OBS'][i][0]
                sid_h_1, grin_mid_1 = one_month(D_1)
                obsrv['DATE-OBS-ONE'][i], obsrv['DATE-OBS-JD'][i] = date_obs(sid_h_1,S1_1,grin_mid_1)
                           
                D_2 = obsrv['DATE-OBS'][i][1]
                sid_h_2, grin_mid_2 = one_month(D_2)
                obsrv['DATE-OBS-ONE_2'][i], obsrv['DATE-OBS-JD_2'][i] = date_obs(sid_h_2,S1_2,grin_mid_2)
                
            # """
            # if we have 1 night (2 dates)
            # """

            elif ';' not in obsrv['DATE-OBS'][i]: 

                if obsrv['TIME-OBS (tзв-s)'][i] == obsrv['TIME-OBS (tзв-s)'][i]: #check if value excist
                    obsrv['TIME-OBS (tзв-s)'][i] = str(obsrv['TIME-OBS (tзв-s)'][i]).replace(',',';')
                    obsrv['TIME-OBS (tзв-s)'][i] = obsrv['TIME-OBS (tзв-s)'][i].split(';')
                    time = obsrv['TIME-OBS (tзв-s)'][i][0]

                    if 's' in time:
                        time = time.replace('s','')
                        time = time.split('h')
                        time_m = time[1].split('m')
                        S1 = float(time[0]) + float(time_m[0])/60 + float(time_m[1])/3600 #local sidereal time (from table)
                        time_sid_table_t = f'{time[0]}:{time_m[0]}:{time_m[1]}'
                        time_sid_table_t = time_sid_table_t.replace(" ","")

                    elif 's' not in time:
                        if 'm' in time:
                            time = time.replace('m','').split('h')
                            S1 = float(time[0]) + float(time[1].split('m')[0])/60 #local sidereal time (from table)

                        elif 'm' not in str(time):
                            time = time.split('h')
                            S1 = float(time[0]) + float(time[1])/60 #local sidereal time (from table)

                    l = str(obsrv['DATE-OBS'][i]).split('.')
                
                    """
                    if there are two month, for example 31.03-01.04.1996
                    """

                    if len(l) > 3:
                        D = obsrv['DATE-OBS'][i]
                        sid_h, grin_mid = two_month(D)
                        obsrv['DATE-OBS-ONE'][i], obsrv['DATE-OBS-JD'][i] = date_obs(sid_h,S1,grin_mid)

                    # """
                    # if we have 1 month 
                    # """

                    else:
                        # """
                        # if we have 1 night (1 date) 01.04.1956
                        # """
                        if '-' not in str(obsrv['DATE-OBS'][i]):
                            Day = str(obsrv['DATE-OBS'][i]).split('.')
                            if len(Day[2]) < 4: #if we have '96' instead of '1996'
                                Day[2] = f'19{Day[2]}'
                            D = f'{Day[2]}-{Day[1]}-{Day[0]}'
                            t = Time(f'{D} 00:00:00')
                            sid_h = t.sidereal_time('mean',longitude=longitude).value*u.hour - 6*u.hour #-6 hour beacouse initially we get GST
                            grin_mid = t - 6*u.hour
                            obsrv['DATE-OBS-ONE'][i], obsrv['DATE-OBS-JD'][i] = date_obs(sid_h,S1,grin_mid)

                        # """
                        # if we have 1 month, for example 02-03.06.1993
                        # """
                        else:
                            D = obsrv['DATE-OBS'][i]
                            sid_h, grin_mid = one_month(D)
                            obsrv['DATE-OBS-ONE'][i], obsrv['DATE-OBS-JD'][i] = date_obs(sid_h,S1,grin_mid)

# """
# RUS ---> ENG
# """

#TELESCOPE

for i,tel in enumerate(obsrv['TELESCOPE']):
    if  tel == '50cm менисковый телескоп Максутова':
        obsrv['TELESCOPE'][i] = 'Wide aperture Maksutov meniscus telescope with main mirror 50 cm'
    elif tel != tel:
        obsrv['TELESCOPE'][i] = 'Wide aperture Maksutov meniscus telescope with main mirror 50 cm'

#OBSERVER

observers_uniq = obsrv['OBSERVER'].unique() #to know each unique observer
#then 
observers_dict = {'Рожковский Д.А.': 'Rozhkovskij D.A.', 'Торопова Т.П.':'Tropova T.P.', 
                  'Городецкий Д.И.':'Gordetskij D.I.', 'Глушков Ю.И.':'Glushkovskij Yu.I.','Торопова Т.П.  Рожковский Д.А.': 'Tropova T.P., Rozhkovskij D.A.',
                  'Рожковский Д.А., Торопова Т.П.' : 'Rozhkovskij D.A., Tropova T.P.', 'Рожковский Д.А., Павлова Л.А.' : 'Rozhkovskij D.A., Pavlova L.A.',
                  'Карягина З.В.':'Karyagina Z.V.', 'Матягин В.С.': 'Matyagin V.S.', 'Павлова Л.А':'Pavlova L.A.', 'Гаврилов':'Gavrilov', 
                  'Курчаков А.В.':'Kurchakov A.V.','Рожковский Д.А.   Городецкий Д.И.':'Rozhkovskij D.A.   Gordetskij D.I.'}
obsrv = obsrv.replace({"OBSERVER":observers_dict})

##  HEADER

In [ ]:
used_rows = [] #number of rows which were used for files
path = 'PATH/TO/DIRECTORY/WITH/FRAMES'

os.chdir(path)

#create new folders to sort frames with full/partly full/doen't be written header
if "no_header" not in os.listdir():
    os.mkdir('no_header')
if "not_full_header" not in os.listdir():
    os.mkdir('not_full_header')
if "full_header" not in os.listdir():
    os.mkdir('full_header')

counter_not_full_header = 0
counter = 0
for f in os.listdir():
    if "fit" in f:
        hdul =  fits.open(f)
        file_name = f.replace('.fits','').replace(".fit","")
        file_name_splitted = file_name.split('_')
        print('---')
        print(f'{f}') 
        for i in range(0,len(obsrv)):
            if file_name_splitted[3] == str(obsrv['Идентификационный номер (номер каробки-номер пластины)'][i]).replace(" ",""):
                print("~~~FIND~~~")
                print(file_name_splitted[3], obsrv['Идентификационный номер (номер каробки-номер пластины)'][i])
                used_rows.append(i)

                if obsrv['Идентификационный номер (номер каробки-номер пластины)'][i] == obsrv['Идентификационный номер (номер каробки-номер пластины)'][i]:
                    hdul[0].header['IDN']=obsrv['Идентификационный номер (номер каробки-номер пластины)'][i]  

                hdul[0].header['OBJECT']=file_name_splitted[0]

                if obsrv['DATE-OBS-ONE'][i] != '': #== obsrv['DATE-OBS-ONE'][i]:
                    hdul[0].header['DATE-OBS']=obsrv['DATE-OBS-ONE'][i] 
                if obsrv['DATE-OBS-ONE_2'][i] != '':
                    hdul[0].header['DATE-OB2']=obsrv['DATE-OBS-ONE_2'][i]    

                try:
                    if obsrv['exposition '][i] == obsrv['exposition '][i]:
                        n = len(obsrv['exposition '][i])                          
                        for g in range(0,n):
                            hdul[0].header[f'EXPTIME{g+1}']=obsrv['exposition '][i][g]                                    
                except TypeError:
                    hdul[0].header[f'EXPTIME']=obsrv['exposition '][i]

                if obsrv['RA'][i] == obsrv['RA'][i]:
                    hdul[0].header['OBJCTRA']=obsrv['RA'][i]                                                   

                if obsrv['DEC   (градусы)'][i] == obsrv['DEC   (градусы)'][i]:
                    hdul[0].header['OBJCTDEC']=obsrv['DEC   (градусы)'][i] 

                hdul[0].header['EPOCH'] = 1950

                if obsrv['DATE-OBS-JD'][i] == obsrv['DATE-OBS-JD'][i]:
                    hdul[0].header['JD']=obsrv['DATE-OBS-JD'][i] 
                if obsrv['DATE-OBS-JD_2'][i] != '':
                    hdul[0].header['JD2']=obsrv['DATE-OBS-JD_2'][i]  

    #             hdul[0].header['AIRMASS'] = 1.0 

                if obsrv['TELESCOPE'][i] == obsrv['TELESCOPE'][i]:
                    hdul[0].header['TELESCOP'] = obsrv['TELESCOPE'][i]          

                if obsrv['OBSERVER'][i] == obsrv['OBSERVER'][i]:
                    hdul[0].header['OBSERVER'] = obsrv['OBSERVER'][i]

                hdul[0].header['OBSERVAT'] = 'FAI'  

                header_keys = list((hdul[0].header).keys())
                if obsrv['DATE-OBS-ONE'][i]=='' or 'OBJCTRA' not in header_keys or 'OBJCTDEC' not in header_keys or 'EXPTIME' not in str(header_keys):
                    counter_not_full_header += 1
                    os.remove(f)
                    os.chdir(path+'/not_full_header')
                    hdul.writeto(f'{file_name}.fits') #SAVE FILES
                    with open('absent_rows.txt', 'a+') as file:
                        file.write(f'\n\n~~~~~ {counter} ~~~~~ \n{file_name}')
                    if obsrv['DATE-OBS-ONE'][i]=='':
                        with open('absent_rows.txt', 'a+') as file:
                            file.write(f'\nNo date of observation (DATE-OBS-ONE)')
                    if 'EXPTIME' not in str(header_keys):
                        with open('absent_rows.txt', 'a+') as file:
                            file.write(f'\nNo exposure time (exposition)')
                    if 'OBJCTRA' not in header_keys:
                        with open('absent_rows.txt', 'a+') as file:
                            file.write(f'\nNo Right Ascension (RA)')
                    if 'OBJCTDEC' not in header_keys:
                        with open('absent_rows.txt', 'a+') as file:
                            file.write(f'\nNo Declination (DEC)')       
                    os.chdir(path)

                else: 
                    os.remove(f)
                    os.chdir(path+'/full_header')
                    hdul.writeto(f'{file_name}.fits') #SAVE FILES
                    os.chdir(path)            
    counter+=1
 
#if id number doesn't excist in the table 
os.chdir(path)
for f in os.listdir(path):
    if "fit.fits" in f:
        shutil.move(os.path.join(path, f), path+'/no_header')